<b>Nicolò Romandini</b></br>
Matricola: 0001023921</br>
PhD Course: A Comparative Introduction to Deep Learning Framework: TensorFlow, PyTorch, JAX.

Dataset Preprocessing

In [32]:
import pandas as pd
import numpy as np
import os


outcomes_a = pd.read_csv('./Outcomes-a.txt')
outcomes_b = pd.read_csv('./Outcomes-b.txt')
outcomes_c = pd.read_csv('./Outcomes-c.txt')
outcomes = pd.concat([outcomes_a, outcomes_b, outcomes_c])

import zipfile
with zipfile.ZipFile("./sets.zip", 'r') as zip_ref:
    zip_ref.extractall(".")

patients = list()
for f in os.listdir('set-a'):
    with open(os.path.join('set-a', f), 'r') as fp:
        txt = fp.readlines()

    txt = [t.rstrip('\n').split(',') for t in txt]
    
    patients.extend([txt[1:]])
for f in os.listdir('set-b'):
    with open(os.path.join('set-b', f), 'r') as fp:
        txt = fp.readlines()
        
    txt = [t.rstrip('\n').split(',') for t in txt]
    
    patients.extend([txt[1:]])
for f in os.listdir('set-c'):
    with open(os.path.join('set-c', f), 'r') as fp:
        txt = fp.readlines()
        
    txt = [t.rstrip('\n').split(',') for t in txt]
    
    patients.extend([txt[1:]])

In [33]:
patients[0][:10]

[['00:00', 'RecordID', '139239'],
 ['00:00', 'Age', '65'],
 ['00:00', 'Gender', '0'],
 ['00:00', 'Height', '-1'],
 ['00:00', 'ICUType', '3'],
 ['00:00', 'Weight', '-1'],
 ['00:01', 'HR', '77'],
 ['00:01', 'NIDiasABP', '69'],
 ['00:01', 'NIMAP', '89'],
 ['00:01', 'NISysABP', '129']]

In [34]:
dataset = []


# The labels of interest = only vital signs.

labels = ['HR', 'NIDiasABP','NISysABP','RespRate', 'NIMAP', 'Temp']


for p in patients:
    temp = np.empty(24)
    temp[:] = np.nan
    patient = {
        'RecordID' : '',
        'HR' : temp.copy(),
        'HR_count' : 0,
        'NIDiasABP' : temp.copy(),
        'NIDiasABP_count': 0,
        'NIMAP': temp.copy(),
        'NIMAP_count' : 0,
        'NISysABP' : temp.copy(),
        'NISysABP_count': 0,
        'RespRate' : temp.copy(),
        'RespRate_count': 0,
        'Temp' : temp.copy(),
        'Temp_count': 0,
        'Outcome' : -1
    }
    
    last_measurment = [-1, -1, -1, -1, -1, -1]

    for measurment in p:
        hour, name, value = measurment
        if name == 'RecordID':
            patient['RecordID'] = value
            pass
        hour = int(hour.split(':')[0])
        value = float(value)

        if name in labels and hour > last_measurment[labels.index(name)] and hour > 23:
            if hour == 49:
                pass
            last_measurment[labels.index(name)] = hour
            if patient[name+'_count'] == 0:
                patient[name][:] = value
            else:
                patient[name][hour%24:] = value
            patient[name+'_count'] += 1
    
    dataset.append(patient)

hr_measurments = []
nidi_measurments = []
nimap_measurments = []
nisis_measurments = []
resprate_measurments = []
temp_measurments = []

filtered_dataset = []

no_features = 0

for patient in dataset:
    if patient['HR_count'] == 0:
        no_features+=1
    if patient['NIDiasABP_count'] == 0:
        no_features+=1
    if patient['NISysABP_count'] == 0: 
        no_features+=1
    if patient['RespRate_count'] == 0:
        no_features+=1
    if patient['NIMAP_count'] == 0:
        no_features+=1
    if patient['Temp_count'] == 0:
        no_features+=1

    # If a patient does not have at least 2 features, he or she is excluded.
    if no_features > 2:
        pass

    hr_measurments.append(patient['HR'])
    nidi_measurments.append(patient['NIDiasABP'])
    nimap_measurments.append(patient['NIMAP'])
    nisis_measurments.append(patient['NISysABP'])
    resprate_measurments.append(patient['RespRate'])
    temp_measurments.append(patient['Temp'])
    patient['Outcome'] = int(outcomes.loc[outcomes['RecordID'] == int(patient['RecordID'])]['In-hospital_death'].values[0])


    filtered_dataset.append(patient)


hr_measurments = np.array(hr_measurments)
nidi_measurments = np.array(nidi_measurments)
nimap_measurments = np.array(nimap_measurments)
nisis_measurments = np.array(nisis_measurments)
resprate_measurments = np.array(resprate_measurments)
temp_measurments = np.array(temp_measurments)

# standardization

hr_measurments = (hr_measurments - np.nanmean(hr_measurments)) / np.nanstd(hr_measurments)
nidi_measurments = (nidi_measurments - np.nanmean(nidi_measurments)) / np.nanstd(nidi_measurments)
nimap_measurments = (nimap_measurments - np.nanmean(nimap_measurments)) / np.nanstd(nimap_measurments) 
nisis_measurments = (nisis_measurments - np.nanmean(nisis_measurments)) / np.nanstd(nisis_measurments)
resprate_measurments = (resprate_measurments - np.nanmean(resprate_measurments)) / np.nanstd(resprate_measurments)
temp_measurments = (temp_measurments - np.nanmean(temp_measurments)) / np.nanstd(temp_measurments)

# nan values are set to 0

hr_measurments[np.isnan(hr_measurments)] = 0
nidi_measurments[np.isnan(nidi_measurments)] = 0
nimap_measurments[np.isnan(nimap_measurments)] = 0
nisis_measurments[np.isnan(nisis_measurments)] = 0
resprate_measurments[np.isnan(resprate_measurments)] = 0
temp_measurments[np.isnan(temp_measurments)] = 0



In [35]:
filtered_dataset[0]

{'RecordID': '139239',
 'HR': array([75., 78., 78., 63., 68., 68., 73., 73., 67., 67., 68., 63., 61.,
        61., 61., 67., 67., 67., 64., 64., 64., 64., 70., 70.]),
 'HR_count': 13,
 'NIDiasABP': array([72., 77., 77., 75., 72., 72., 70., 70., 73., 73., 63., 63., 79.,
        77., 77., 66., 66., 66., 82., 82., 82., 82., 76., 76.]),
 'NIDiasABP_count': 12,
 'NIMAP': array([ 99.33, 101.  , 101.  ,  99.  , 101.3 , 101.3 ,  95.33,  95.33,
         99.33,  99.33,  88.67,  88.67, 107.  , 105.7 , 105.7 ,  93.67,
         93.67,  93.67, 111.  , 111.  , 111.  , 111.  , 103.3 , 103.3 ]),
 'NIMAP_count': 12,
 'NISysABP': array([154., 149., 149., 147., 160., 160., 146., 146., 152., 152., 140.,
        140., 163., 163., 163., 149., 149., 149., 169., 169., 169., 169.,
        158., 158.]),
 'NISysABP_count': 12,
 'RespRate': array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]),
 'RespRate_count': 0,
 'Temp': array([3

In [36]:
index = 0

X = []
y = []

for patient in filtered_dataset:
    arr = np.array([hr_measurments[index], nidi_measurments[index], nimap_measurments[index], nisis_measurments[index], resprate_measurments[index], temp_measurments[index]])
    rot = np.rot90(arr, k=3)
    X.append(rot.tolist())
    y.append(patient['Outcome'])
    index += 1


In [37]:
X

[[[-0.5547454914334333,
   0.0,
   1.3893628718309636,
   1.4826610095628359,
   0.9494127187353737,
   -0.6425182251723366],
  [-0.5547454914334333,
   0.0,
   1.195068064272987,
   1.593357525793978,
   1.2725624496718013,
   -0.47137250036435246],
  [-0.5547454914334333,
   0.0,
   1.195068064272987,
   1.593357525793978,
   1.2725624496718013,
   -0.47137250036435246],
  [-0.5547454914334333,
   0.0,
   1.1173501412497964,
   1.4607868476728498,
   1.1433025572972302,
   -1.327101124404273],
  [-0.5547454914334333,
   0.0,
   1.6225166409005354,
   1.613243127512147,
   0.9494127187353737,
   -1.0418582497242994],
  [-0.5547454914334333,
   0.0,
   1.6225166409005354,
   1.613243127512147,
   0.9494127187353737,
   -1.0418582497242994],
  [-0.293050904104728,
   0.0,
   1.078491179738201,
   1.2175196533205794,
   0.8201528263608027,
   -0.756615375044326],
  [-0.293050904104728,
   0.0,
   1.078491179738201,
   1.2175196533205794,
   0.8201528263608027,
   -0.756615375044326],
  [

In [38]:
y

[1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,


In [39]:

import pandas as pd
from sklearn.model_selection import train_test_split


In [40]:
from imblearn.under_sampling import RandomUnderSampler

#  classes are highly unbalanced so I simply undersample the majority class

# I need to reshape the array 
new_X = []
for x in X:
    x = np.array(x)
    x = x.flatten()
    new_X.append(x.tolist())

us = RandomUnderSampler(random_state=42)
new_X, new_y = us.fit_resample(new_X, y)

In [41]:
# reshape in the original form

X_os = []
for x in new_X:
    x = np.array(x)
    x = x.reshape((24,6))
    X_os.append(x.tolist())
y_os = new_y

In [42]:
X = X_os
y = y_os

In [43]:
# split train and test set

X_train , X_test , y_train , y_test  = train_test_split(X , y , test_size = 0.25, stratify=y)

<b>TensorFlow<b>

In [44]:

import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv1D, Flatten, Activation

tf.random.set_seed(42)

In [45]:

keras.backend.clear_session()
nn_model_tensorflow = Sequential()
nn_model_tensorflow.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(24,6)))
nn_model_tensorflow.add(Dropout(0.5))
nn_model_tensorflow.add(Conv1D(filters=16, kernel_size=3, activation='relu'))
nn_model_tensorflow.add(Dropout(0.5))
nn_model_tensorflow.add(Flatten())
nn_model_tensorflow.add(Dense(1, activation='sigmoid'))

In [46]:
nn_model_tensorflow.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 22, 32)            608       
                                                                 
 dropout (Dropout)           (None, 22, 32)            0         
                                                                 
 conv1d_1 (Conv1D)           (None, 20, 16)            1552      
                                                                 
 dropout_1 (Dropout)         (None, 20, 16)            0         
                                                                 
 flatten (Flatten)           (None, 320)               0         
                                                                 
 dense (Dense)               (None, 1)                 321       
                                                                 
Total params: 2,481
Trainable params: 2,481
Non-trainabl

In [47]:
opt = tf.keras.optimizers.Adam()

In [48]:
nn_model_tensorflow.compile(loss="binary_crossentropy", optimizer=opt, metrics='accuracy')

In [49]:
from keras.callbacks import ReduceLROnPlateau
lr_reduce = ReduceLROnPlateau(monitor="accuracy", factor=0.1, min_delta=0.0001, patience=10 )

In [50]:
history = nn_model_tensorflow.fit(X_train, y_train , epochs=100, callbacks=[lr_reduce])

Epoch 1/100
80/80 [==============================] - 1s 5ms/step - loss: 0.7175 - accuracy: 0.5066 - lr: 0.0010
Epoch 2/100
80/80 [==============================] - 0s 5ms/step - loss: 0.6974 - accuracy: 0.5262 - lr: 0.0010
Epoch 3/100
80/80 [==============================] - 0s 5ms/step - loss: 0.6767 - accuracy: 0.5570 - lr: 0.0010
Epoch 4/100
80/80 [==============================] - 0s 5ms/step - loss: 0.6778 - accuracy: 0.5703 - lr: 0.0010
Epoch 5/100
80/80 [==============================] - 0s 5ms/step - loss: 0.6656 - accuracy: 0.5949 - lr: 0.0010
Epoch 6/100
80/80 [==============================] - 0s 5ms/step - loss: 0.6535 - accuracy: 0.6066 - lr: 0.0010
Epoch 7/100
80/80 [==============================] - 0s 5ms/step - loss: 0.6572 - accuracy: 0.5973 - lr: 0.0010
Epoch 8/100
80/80 [==============================] - 0s 5ms/step - loss: 0.6573 - accuracy: 0.6078 - lr: 0.0010
Epoch 9/100
80/80 [==============================] - 0s 5ms/step - loss: 0.6511 - accuracy: 0.6102 - lr:

In [51]:
results = nn_model_tensorflow.evaluate(X_test, y_test)
print("test loss, test acc:", results)

27/27 [==============================] - 0s 3ms/step - loss: 0.7001 - accuracy: 0.6054
test loss, test acc: [0.7000550627708435, 0.6053864359855652]


<p>PyTorch</b>

In [52]:

import torch.nn as nn
import torch

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.layers = nn.Sequential(
            nn.Conv1d(24,32,3),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Conv1d(32,16,3),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Flatten(),
            nn.Linear(32,1),
            nn.Sigmoid()
        )

    

    def forward(self, x):
        x = self.layers(x)
        x = x[:, -1]

        return x

In [53]:
X_train_tensor = torch.tensor(X_train)
y_train_tensor = torch.tensor(y_train)

# need to cast to use BCELoss
y_train_tensor = y_train_tensor.type(torch.float)

In [54]:
X_train_tensor

tensor([[[-0.3803,  1.3618,  1.1174,  1.7723,  1.5957, -0.5855],
         [-0.3803,  0.2525,  0.5733,  1.4164,  1.5311, -0.6425],
         [-0.3803,  0.4374,  0.6122,  1.5271,  1.6603, -0.8137],
         ...,
         [-0.1186,  0.4374,  0.1459,  0.6872,  0.8202, -0.6425],
         [-0.1186,  0.0676,  0.4179,  0.7979,  0.7555, -0.8137],
         [-0.1186,  0.9921,  0.4567,  1.1738,  1.2726, -0.6425]],

        [[ 0.4048,  1.9165,  0.0000,  0.0000,  0.0000,  1.6394],
         [ 0.4048,  3.0258,  0.0000,  0.0000,  0.0000,  1.5824],
         [ 0.4048,  1.5467,  0.0000,  0.0000,  0.0000,  2.4952],
         ...,
         [-0.3803,  0.4374,  0.0000,  0.0000,  0.0000,  1.8106],
         [-0.3803,  0.4374,  0.0000,  0.0000,  0.0000,  1.8106],
         [ 0.4920,  1.5467,  0.0000,  0.0000,  0.0000,  0.8978]],

        [[ 0.5793,  0.0000,  1.5059,  1.7723,  1.2726,  0.7837],
         [ 0.5793,  0.0000,  1.5059,  1.7723,  1.2726,  0.6696],
         [ 0.5793,  0.0000,  1.5059,  1.7723,  1.2726,  0.

In [55]:
nn_model_pytorch = Net()
print(nn_model_pytorch.to("cpu"))

Net(
  (layers): Sequential(
    (0): Conv1d(24, 32, kernel_size=(3,), stride=(1,))
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
    (3): Conv1d(32, 16, kernel_size=(3,), stride=(1,))
    (4): ReLU()
    (5): Dropout(p=0.5, inplace=False)
    (6): Flatten(start_dim=1, end_dim=-1)
    (7): Linear(in_features=32, out_features=1, bias=True)
    (8): Sigmoid()
  )
)


In [56]:

import torch.optim as optim


opt = optim.Adam(nn_model_pytorch.parameters())
scheduler = optim.lr_scheduler.ReduceLROnPlateau(opt, mode='min', factor=0.1, patience=10, threshold=0.0001, threshold_mode='abs')
for _ in range(100):
    loss_fn = nn.BCELoss()
    nn_model_pytorch.train()
    opt.zero_grad()
    pred = nn_model_pytorch(X_train_tensor)
    loss = loss_fn(pred, y_train_tensor)
    loss.backward()       
    opt.step()
    scheduler.step(loss)
    loss = loss.item()

In [57]:
X_test_tensor = torch.tensor(X_test)
y_test_tensor = torch.tensor(y_test)
y_test_tensor = y_test_tensor.type(torch.float)

In [61]:
loss_fn = nn.BCELoss()
nn_model_pytorch.eval()
with torch.no_grad():
    output = nn_model_pytorch(X_test_tensor)
    loss = loss_fn(output, y_test_tensor).item()
    print(loss)


0.6792199015617371
